In [4]:
# -*- coding: utf-8 -*-
"""
하루 단위 → 시간 단위(시/스케줄 블록) 공정 시뮬레이션 (Press 과대 방지 캘리브레이션)
- 입력 CSV: subset_20240101_20250630_fullcols.csv
- 출력 CSV: FinalResults_time_scheduled_20240101_20250630.csv
- Little's Law 기반 분해 후,
  (1) 우선순위 가중치 → (2) 원본 퍼센타일 상한 캡 → (3) 일평균 보정(재정규화)
"""

import numpy as np
import pandas as pd

# =========================
# 1) CONFIG
# =========================
INPUT_CSV   = "./subset_20240101_20250630_fullcols.csv"
OUTPUT_CSV  = "./FinalResults_time_scheduled_20240101_20250630.csv"
GRANULARITY = "hourly"  # "hourly" or "block"

# 스케줄 (07:50 ~ 다음날 07:40, 20:10~20:40 휴식 포함)
SCHEDULE_BLOCKS = [
    ("07:50","09:50","주간"),
    ("09:50","10:00","휴식"),
    ("10:00","11:50","주간"),
    ("11:50","12:40","중식"),
    ("12:40","14:40","주간"),
    ("14:40","14:50","휴식"),
    ("14:50","16:40","주간"),
    ("16:40","17:10","석식"),
    ("17:10","20:10","잔업"),
    ("20:10","20:40","휴식"),
    ("20:40","00:00","야간"),
    ("00:00","00:30","야식"),
    ("00:30","04:00","야간"),
    ("04:00","05:00","휴식"),
    ("05:00","07:40","야간"),
]

# 근무/휴식 가중치 (생산 분배 및 Util/Time 변조)
SHIFT_WEIGHTS = {"주간":1.0, "잔업":0.9, "야간":1.2, "중식":0.0, "석식":0.0, "야식":0.0, "휴식":0.0}
UTIL_BAND = 0.15     # ±15%
TIME_BAND = 0.10     # ±10%

# === 병목 순위 반영용 우선순위 가중치(Press 축소) ===
#   - 상위 두 개는 살짝 ↑, 중하위는 1.0 근처, Press류는 <1로 ↓
CALIBRATION_WEIGHTS = {
    "Warehouse1_Queue":        1.20,
    "Forklift_Blanking_Queue": 1.10,
    "Warehouse_3_Queue":       1.05,
    "Warehouse_4_Queue":       1.02,
    "Forklift_Press_Queue":    0.95,
    "Press4_Queue":            0.90,
    "Press2_Queue":            0.88,
    "Blanking_SKU3_Queue":     0.98,
    # 지정 외 나머지 기본값:
    "_default_":               0.95
}

# === 원본 기반 상한 캡(퍼센타일) ===
PCTL_FOR_CAP = 95  # 90/95/99 등 조정 가능
CAP_MULTIPLIER = 1.00  # pctl×1.0이 상한. 약간 여유 주려면 1.1 등

# =========================
# 2) HELPERS
# =========================
def to_dt(s):
    try: return pd.to_datetime(s, errors="coerce")
    except: return pd.to_datetime(s.astype(str), errors="coerce")

def detect_date_col(df: pd.DataFrame) -> str:
    for c in ["Time_Now", "Timestamp", "timestamp", "Date", "date"]:
        if c in df.columns: return c
    return df.columns[0]

def day_floor(dt: pd.Series) -> pd.Series:
    return dt.dt.floor("D")

def aggregate_to_daily(df: pd.DataFrame, date_col: str):
    """일 단위 집계(생산 sum / 시간·유틸·큐 mean / 나머지 first)"""
    dt = to_dt(df[date_col])
    if dt.isna().all():
        raise ValueError(f"Could not parse datetime from '{date_col}'")
    df = df.copy()
    df["_date"] = day_floor(dt)

    count_cols = [c for c in df.columns if c.startswith("c_Cell") or c == "c_TotalProducts"]
    time_cols  = [c for c in df.columns if c.endswith("_Time")]
    util_cols  = [c for c in df.columns if c.endswith("_Util")]
    queue_cols = [c for c in df.columns if c.endswith("_Queue")]
    other_cols = sorted(set(df.columns) - set([date_col, "_date"] + count_cols + time_cols + util_cols + queue_cols))

    agg = {}
    for c in count_cols: agg[c] = "sum"
    for c in time_cols + util_cols + queue_cols: agg[c] = "mean"
    for c in other_cols: agg[c] = "first"

    daily = df.groupby("_date", as_index=False).agg(agg).sort_values("_date").reset_index(drop=True)
    daily[date_col] = daily["_date"]
    return daily.drop(columns=["_date"]), count_cols, time_cols, util_cols, queue_cols, date_col

def build_schedule_for_day(day: pd.Timestamp) -> pd.DataFrame:
    rows = []
    anchor_start = pd.Timestamp(day.year, day.month, day.day, 7, 50)
    anchor_end   = pd.Timestamp(day.year, day.month, day.day, 7, 40) + pd.Timedelta(days=1)

    for start_str, end_str, label in SCHEDULE_BLOCKS:
        s_h, s_m = map(int, start_str.split(":"))
        e_h, e_m = map(int, end_str.split(":"))
        s = pd.Timestamp(day.year, day.month, day.day, s_h, s_m)
        e = pd.Timestamp(day.year, day.month, day.day, e_h, e_m)
        if (s_h, s_m) < (7,50): s += pd.Timedelta(days=1)
        if (e_h, e_m) < (7,50) or e <= s: e += pd.Timedelta(days=1)
        s = max(s, anchor_start); e = min(e, anchor_end)
        if e <= s: 
            continue
        minutes = int((e - s).total_seconds() // 60)
        rows.append({"start": s, "end": e, "label": label, "minutes": minutes})
    return pd.DataFrame(rows)

def explode_to_hours(sch: pd.DataFrame) -> pd.DataFrame:
    t_min = sch["start"].min().floor("h")
    t_max = sch["end"].max().ceil("h")
    hours = pd.date_range(t_min, t_max, freq="h")
    slots = pd.DataFrame({"slot_start": hours[:-1], "slot_end": hours[1:]})
    out = []
    for _, blk in sch.iterrows():
        for _, sl in slots.iterrows():
            s = max(blk["start"], sl["slot_start"])
            e = min(blk["end"], sl["slot_end"])
            overlap = (e - s).total_seconds() / 60.0
            if overlap > 0:
                out.append({
                    "slot_start": sl["slot_start"],
                    "slot_end": sl["slot_end"],
                    "label": blk["label"],
                    "min_overlap": overlap
                })
    return pd.DataFrame(out)

def integer_allocate(total: float, weights: np.ndarray) -> np.ndarray:
    w = np.clip(np.asarray(weights, float), 0, None)
    if w.sum() <= 0 or total <= 0: return np.zeros_like(w, dtype=int)
    p = w / w.sum()
    alloc = np.floor(total * p).astype(int)
    residual = int(round(total - alloc.sum()))
    if residual > 0:
        idx = np.argsort(-p)[:residual]
        alloc[idx] += 1
    return alloc

# =========================
# 3) MAIN TRANSFORM
# =========================
def make_time_scheduled(df_daily: pd.DataFrame,
                        count_cols, time_cols, util_cols, queue_cols,
                        date_col: str,
                        granularity: str,
                        global_caps: dict) -> pd.DataFrame:
    rows = []

    # SKU별 Wait 컬럼 매핑 (예: "SKU3_Wait_Time")
    sku_wait_cols = {c.split("_")[0]: c for c in time_cols if c.startswith("SKU") and c.endswith("Wait_Time")}
    overall_wait_name = next((c for c in time_cols if c.endswith("Wait_Time")), None)

    for _, day_row in df_daily.iterrows():
        day = pd.to_datetime(day_row[date_col]).floor("D")
        sch = build_schedule_for_day(day)
        if sch.empty:
            continue

        if granularity == "block":
            slot_df = sch.rename(columns={"start":"slot_start","end":"slot_end"})
            slot_df["min_overlap"] = (slot_df["slot_end"] - slot_df["slot_start"]).dt.total_seconds()/60.0
            slot_df = slot_df[["slot_start","slot_end","label","min_overlap"]]
        else:
            slot_df = explode_to_hours(sch)

        # 생산 가중치
        slot_df["shift_weight"] = slot_df["label"].map(SHIFT_WEIGHTS).fillna(0.0)
        slot_df["work_minutes"] = slot_df["min_overlap"] * (slot_df["shift_weight"] > 0).astype(float)
        slot_df["prod_weight"] = slot_df["work_minutes"] * slot_df["shift_weight"]

        # count 분배(정수)
        for c in count_cols:
            total = float(day_row[c])
            slot_df[c] = integer_allocate(total, slot_df["prod_weight"].values)

        # Util/Time 분배
        safe_eps = 1e-12
        work_mask = slot_df["shift_weight"].values > 0
        sw = slot_df["shift_weight"].values.copy()
        if work_mask.any():
            sw_norm = np.zeros_like(sw, dtype=float)
            mean_sw = sw[work_mask].mean()
            sw_norm[work_mask] = sw[work_mask] / (mean_sw + safe_eps)
        else:
            sw_norm = np.zeros_like(sw, dtype=float)

        for c in util_cols:
            mean_u = float(day_row[c])
            u = np.zeros(len(slot_df), dtype=float)
            u[work_mask] = mean_u * (1.0 - UTIL_BAND + 2*UTIL_BAND * sw_norm[work_mask])
            slot_df[c] = np.clip(u, 0, 1)

        for c in time_cols:
            mean_t = float(day_row[c])
            t = np.zeros(len(slot_df), dtype=float)
            if mean_t != 0 and work_mask.any():
                t[work_mask] = mean_t * (1.0 - TIME_BAND + 2*TIME_BAND * sw_norm[work_mask])
            slot_df[c] = t

        # Queue(L = λ×W) — 1차 계산
        slot_df["slot_seconds"] = (slot_df["slot_end"] - slot_df["slot_start"]).dt.total_seconds()
        per_slot_total = slot_df[count_cols].sum(axis=1).astype(float)
        lam = (per_slot_total / slot_df["slot_seconds"].replace(0, np.nan)).fillna(0.0)

        for qc in queue_cols:
            daily_mean_q = float(day_row[qc])

            # SKUx 매칭
            sku_tag = next((p for p in qc.split("_") if p.startswith("SKU")), None)
            if sku_tag and sku_tag in sku_wait_cols:
                inferred_W = slot_df[sku_wait_cols[sku_tag]].astype(float)
            elif overall_wait_name is not None:
                inferred_W = slot_df[overall_wait_name].astype(float)
            else:
                inferred_W = pd.Series(np.zeros(len(slot_df)))

            L = lam * inferred_W
            if (L > 0).any():
                scale = (daily_mean_q / (L.mean() + 1e-9)) if daily_mean_q != 0 else 0.0
                q_vals = (L * scale).values
            else:
                w = slot_df["prod_weight"].values
                w = w / (w.sum() + 1e-9)
                q_vals = w * daily_mean_q * len(slot_df)

            # ---------- (A) 우선순위 가중치 적용 ----------
            weight = CALIBRATION_WEIGHTS.get(qc, CALIBRATION_WEIGHTS["_default_"])
            q_vals = q_vals * weight

            # ---------- (B) 원본 기반 상한 캡 ----------
            cap = global_caps.get(qc, None)
            if cap is not None:
                q_vals = np.minimum(q_vals, cap)

            slot_df[qc] = q_vals

        # ---------- (C) 일평균 보정(재정규화) ----------
        #   우선순위/캡 적용으로 큐 평균이 변했을 수 있으므로, 각 큐별로
        #   (원래 일평균)과 (현재 일평균)을 맞춰주기 위한 스케일을 한 번 더 적용
        for qc in queue_cols:
            if qc not in slot_df.columns:
                continue
            orig_mean = float(day_row[qc])
            cur_mean = float(slot_df[qc].mean()) if len(slot_df) > 0 else 0.0
            if cur_mean > 0 and orig_mean > 0:
                slot_df[qc] *= (orig_mean / cur_mean)
            elif cur_mean == 0 and orig_mean > 0:
                # 모두 0이 돼버렸다면 생산 가중치비로 적분치 보정
                w = slot_df["prod_weight"].values
                w = w / (w.sum() + 1e-9)
                slot_df[qc] = w * orig_mean * len(slot_df)

        # 출력 누적
        for _, r in slot_df.iterrows():
            out = {"Time_Now": r["slot_start"], "Time_End": r["slot_end"], "ShiftLabel": r["label"]}
            for c in count_cols + util_cols + time_cols + queue_cols:
                out[c] = r.get(c, 0)
            rows.append(out)

    result = pd.DataFrame(rows).sort_values(["Time_Now"]).reset_index(drop=True)

    # 컬럼 순서 정리
    keep_order, all_cols = [], set(count_cols + util_cols + time_cols + queue_cols)
    for c in df_daily.columns:
        if c in all_cols: keep_order.append(c)
    for c in (set(all_cols) - set(keep_order)): keep_order.append(c)
    result = result.reindex(columns=["Time_Now","Time_End","ShiftLabel"] + keep_order)
    return result


# =========================
# 4) RUN
# =========================
if __name__ == "__main__":
    # 1) 로드
    full = pd.read_csv(INPUT_CSV, low_memory=False)

    # 2) 날짜 컬럼 탐지 및 datetime 변환
    date_col = detect_date_col(full)
    full[date_col] = to_dt(full[date_col])
    if full[date_col].isna().all():
        raise ValueError(f"날짜 파싱 실패: '{date_col}' 컬럼을 datetime으로 변환할 수 없습니다.")
    full["_date"] = full[date_col].dt.floor("D")

    # 3) 열 그룹화
    count_cols = [c for c in full.columns if c.startswith("c_Cell") or c == "c_TotalProducts"]
    time_cols  = [c for c in full.columns if c.endswith("_Time")]
    util_cols  = [c for c in full.columns if c.endswith("_Util")]
    queue_cols = [c for c in full.columns if c.endswith("_Queue")]

    # 4) 원본 전체에서 큐별 상한 캡(퍼센타일) 계산
    global_caps = {}
    for qc in queue_cols:
        try:
            pctl = np.nanpercentile(full[qc].astype(float).values, PCTL_FOR_CAP)
            global_caps[qc] = float(pctl) * CAP_MULTIPLIER
        except Exception:
            global_caps[qc] = None  # 계산 불가시 캡 미적용

    # 5) 입력이 '하루 1행'인지 판단 → 일 집계/스킵
    nunique_dates = full["_date"].nunique()
    is_daily_like = (len(full) == nunique_dates)

    if is_daily_like:
        df_daily = full.drop(columns=["_date"]).copy()
        df_daily[date_col] = full["_date"].values
    else:
        df_daily = full.drop(columns=["_date"]).copy()
        df_daily, count_cols, time_cols, util_cols, queue_cols, date_col = aggregate_to_daily(df_daily, date_col)

    # 6) 시간표 기반 분해 + 캘리브레이션/캡/보정
    out = make_time_scheduled(
        df_daily,
        count_cols=count_cols,
        time_cols=time_cols,
        util_cols=util_cols,
        queue_cols=queue_cols,
        date_col=date_col,
        granularity=GRANULARITY,
        global_caps=global_caps
    )

    # 7) 저장
    out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

    # 8) 간단 진단: 대표 두 날짜의 "시간대 최댓값 큐" 빈도
    for tgt in ["2025-03-15","2025-06-30"]:
        d = pd.Timestamp(tgt)
        one = out[out["Time_Now"].dt.floor("D")==d]
        if one.empty: 
            print(f"[디버그] {tgt} 데이터 없음"); 
            continue
        # 시간별 최댓값 큐 집계
        qmax = one[queue_cols].astype(float).idxmax(axis=1)
        print(f"\n[디버그] {tgt} 시간대 최댓값 큐 상위 10개:")
        print(qmax.value_counts().head(10))

    print(f"\n[OK] Saved -> {OUTPUT_CSV}")
    print(f"입력 날짜 수: {nunique_dates:,} | 실제 행수 = {len(out):,}")


[디버그] 2025-03-15 시간대 최댓값 큐 상위 10개:
Warehouse1_Queue           25
Blanking_SKU1_Queue         9
Forklift_Blanking_Queue     1
Name: count, dtype: int64

[디버그] 2025-06-30 시간대 최댓값 큐 상위 10개:
Warehouse1_Queue           24
Blanking_SKU1_Queue         9
Forklift_Blanking_Queue     2
Name: count, dtype: int64

[OK] Saved -> ./FinalResults_time_scheduled_20240101_20250630.csv
입력 날짜 수: 547 | 실제 행수 = 19,145
